In [52]:
import warnings
warnings.filterwarnings("ignore")

In [53]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
import os
from IPython.display import Markdown

In [54]:
os.environ["AWS_ACCESS_KEY_ID"] = os.environ.get("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.environ.get("AWS_SECRET_ACCESS_KEY")
os.environ["AWS_REGION_NAME"] = "us-east-1"

In [ ]:
# @tool("Tool Name")                           
# def my_simple_tool(question: str) -> str:           
#     """Tool description for clarity."""           
#     # Tool logic here                       
#     return "Tool output"                         

In [55]:
@tool('DuckDuckGoSearch')
def search(search_query: str):
    """Search the web for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

In [56]:
@tool('CalculatorAdd')
def calculator_add(a,b):
    """Adds two numbers and gives a result"""
    return (a+b)

In [57]:
modelId="bedrock/anthropic.claude-3-haiku-20240307-v1:0"

In [58]:
# Configure the LLM
llm = LLM(model=modelId)

In [59]:
# Define the Agent
router_agent = Agent(
    role='Router Assistant',
    goal='Assistant to route to the appropriate tool',
    backstory="You are an experienced assistant specializing in routing the flow to the appropriate tool.",
    verbose=False,
    allow_delegation=False,
    llm=llm,
    tools=[search, calculator_add]  
)

In [60]:
# Define the Task
task = Task(
    description="Based on the user's questions: {questions}, route to the appropriate tool",
    expected_output="A properly well worded answer from the tool which can be used by the user.",
    agent=router_agent
)

In [61]:
# Create the Crew
crew = Crew(
    agents=[router_agent],
    tasks=[task],
    verbose=False,
)

2025-03-28 21:07:52,845 - 17988 - __init__.py-__init__:535 - WARNING: Overriding of current TracerProvider is not allowed


In [62]:
# User input for travel preferences
user_input = {
    "questions": "What is 7+9"
}

# Execute the Crew
result = crew.kickoff(inputs=user_input)

In [63]:
Markdown(result.raw)

7 + 9 = 16.

In [64]:
# Enabling Memory in the Agent
crew_with_memory = Crew(
    agents=[router_agent],
    tasks=[task],
    verbose=True,
    memory=True,  # Enable memory
    embedder={
        "provider": "aws_bedrock",
        "config": {
            "model": "amazon.titan-embed-text-v1",  # Embedding model for memory
            "vector_dimension": 1024
        }
    },
    
)

2025-03-28 21:08:04,045 - 17988 - __init__.py-__init__:535 - WARNING: Overriding of current TracerProvider is not allowed


In [65]:
# Executing the Crew with Memory
result_with_memory = crew_with_memory.kickoff(inputs=user_input)

# Agent: Router Assistant
## Task: Based on the user's questions: What is 7+9, route to the appropriate tool


# Agent: Router Assistant
## Thought: Thought: To find the answer to "What is 7 + 9", I should use the CalculatorAdd tool.
## Using tool: CalculatorAdd
## Tool Input: 
"{}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: calculator_add() missing 2 required positional arguments: 'a' and 'b'.
 Tool CalculatorAdd accepts these inputs: CalculatorAdd() - Adds two numbers and gives a result .
Moving on then. I MUST either use a tool (use one at time) OR give my best final answer not both at the same time. To Use the following format:

Thought: you should always think about what to do
Action: the action to take, should be one of [DuckDuckGoSearch, CalculatorAdd]
Action Input: the input to the action, dictionary enclosed in curly braces
Observation: the result of the action
... (this Thought/Action/Action Input/Result can repeat N times)
Thou

In [66]:
from IPython.display import Markdown

In [67]:
Markdown(result_with_memory.raw)

The answer to "What is 7 + 9?" is 16.

In [68]:
user_input = {
    "questions": "What is the result of the match between GT vs MI in IPL ?"
}

# Execute the Crew
result = crew.kickoff(inputs=user_input)

In [69]:
Markdown(result.raw)

According to the information found, the recent head-to-head record between Gujarat Titans (GT) and Mumbai Indians (MI) in the IPL is as follows:

- GT won by 6 runs
- GT won by 62 runs 
- MI won by 27 runs
- GT won by 55 runs
- MI won by 5 runs

The latest match between the two teams was played in IPL 2025, with GT emerging victorious. The key player battles were between Mumbai Indians captain Rohit Sharma and Kagiso Rabada of the Gujarat Titans. Overall, the GT vs MI rivalry has been closely contested, with Gujarat holding the edge in recent matches.

In [70]:
user_input = {
    "questions": "Get the age of Aamir Khan of Bollywood and add 60 to it"
}

# Execute the Crew
result = crew.kickoff(inputs=user_input)

In [71]:
Markdown(result.raw)

Based on the information gathered, Aamir Khan's current age is 58 years old. To get the age after adding 60 years, the calculation would be:

Aamir Khan's current age: 58 years
Add 60 years: 58 + 60 = 118 years old

So the final answer is that Aamir Khan's age after adding 60 years to his current age of 58 would be 118 years old.